## Legal Violations

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Data Directory
dir_original_files = './04-estacoes-selecionadas/'
ficheiro_medicoes = dir_original_files + 'medicoes-longo_seleccionadas.csv'  
# Destination Directory
dir_destination = './06-violacoes-legais/'

In [ ]:
# Measure file
medicoes = pd.read_csv(ficheiro_medicoes, thousands=',', index_col=0, parse_dates=True)
medicoes

In [ ]:
# Create pollutant list
lista_poluentes = medicoes['Poluente'].unique()
lista_poluentes

In [ ]:
# Create day and year column
medicoes['Ano'] = medicoes.index.year
medicoes['Dia'] = medicoes.index.date
medicoes

## First violation: PM2.5

PM2.5 – A1: Measurement period of 1 year; the limit value until 2015 was 25 µg/m³. The number of permitted exceedances per year is not defined.

PM2.5 – A2: Measurement period of 1 year; the limit value until 2020 was 20 µg/m³. The number of permitted exceedances per year is not defined.



In [ ]:
# Filter only PM25 pollutant measures
medicoes_PM25 = medicoes[medicoes['Poluente'] == 'PM2.5']
medicoes_PM25

In [ ]:
# Visualize the average values for each year and for each station
valor_medio_ano_PM25 = medicoes_PM25.groupby(['Station','Ano'])['Valor'].mean().reset_index(name='Média do Ano')
valor_medio_ano_PM25

In [ ]:
# Function that checks for violation
def violacao(clausula, valor):
    if clausula == 'A1':
        if valor > 25:        # limit = 25 until 2015
            return 'Violação'
    elif clausula == 'A2':   # limit = 20 starting in 2020
        if valor > 20:
            return 'Violação'
    return 'Válido'

In [ ]:
# We will apply only clause A2, therefore, the maximum value must be up to 20 µg/m³
clausula = 'A2'

In [ ]:
# Insert violations column
valor_medio_ano_PM25['Violação Legal'] = valor_medio_ano_PM25['Média do Ano'].apply(lambda x: violacao(clausula,x))
valor_medio_ano_PM25

In [ ]:
# Remove valid entries
violacoes_PM25 = valor_medio_ano_PM25.loc[valor_medio_ano_PM25['Violação Legal'] == 'Violação']
violacoes_PM25

In [ ]:
# Save as CSV
violacoes_PM25.to_csv(dir_destination + 'violacoes_pm25.csv', index=False) 

## Second violation: PM10

PM10 – D: Measurement period is 24 hours (1 day); daily mean limit value is 50.  
Number of permitted exceedances per year: 35.

PM10 – A: Measurement period is annual; annual mean limit value is 40.

In [ ]:
# Filter measurements for the PM10 pollutant only
medicoes_PM10 = medicoes[medicoes['Poluente'] == 'PM10']
medicoes_PM10

In [ ]:
# Compute the annual mean to check PM10 – A
medicoes_PM10_violacao_anual = medicoes_PM10.groupby(['Station', 'Ano'])['Valor'].mean().reset_index(name = 'Média Anual')  
medicoes_PM10_violacao_anual

In [ ]:
# Violations based on annual measurements
medicoes_PM10_violacao_anual['Violação Anual'] = medicoes_PM10_violacao_anual['Média Anual'].apply(lambda x: 'Violação' if x > 40 else 'Válido')
medicoes_PM10_violacao_anual

In [ ]:
# Compute the daily mean to check PM10 – D
medicoes_PM10_violacao_diaria = medicoes_PM10.groupby(['Station', 'Dia', 'Ano'])['Valor'].mean().reset_index(name='Média Diária')  
medicoes_PM10_violacao_diaria

In [ ]:
# Violations based on daily measurements
medicoes_PM10_violacao_diaria = medicoes_PM10_violacao_diaria.loc[medicoes_PM10_violacao_diaria['Média Diária'] > 50]
medicoes_PM10_violacao_diaria = medicoes_PM10_violacao_diaria.groupby(['Station', 'Ano'])['Média Diária'].size().reset_index(name = 'Excedências Diárias')
medicoes_PM10_violacao_diaria

In [ ]:
# Count the number of days per year with violations
medicoes_PM10_violacao_diaria['Violação Dias'] = medicoes_PM10_violacao_diaria['Excedências Diárias'].apply(lambda x: 'Violação' if x > 35 else 'Válido')
medicoes_PM10_violacao_diaria

In [ ]:
# Merge both analysis DataFrames
medicoes_PM10_final = medicoes_PM10_violacao_anual[['Station', 'Ano','Média Anual']].merge(
    medicoes_PM10_violacao_diaria[['Station', 'Ano', 'Excedências Diárias']], on = ['Station', 'Ano'], how = 'outer')

medicoes_PM10_final

In [ ]:
# Check whether there are years without daily exceedances
print('Existem valores nulos:')
print(medicoes_PM10_final.isna().values.any())

In [ ]:
# Properly handle missing values: they occur because there were no exceedances that year at that station
medicoes_PM10_final = medicoes_PM10_final.fillna(0)
print('Existem valores nulos:')
print(medicoes_PM10_final.isna().values.any())

In [ ]:
# Projct the DataFrame
medicoes_PM10_final

In [ ]:
# Function that classifies the detected violations
def classificacao_violacao(media, excedencias):
    if media > 40 and excedencias > 35: 
       return 'Ambos'
    elif media > 40:
        return 'Média'
    elif excedencias > 35:
        return 'Dias'
    else: 
        return 'Válido'

In [ ]:
# Add a violation classification column
medicoes_PM10_final['Motivo'] = medicoes_PM10_final.apply(lambda x: classificacao_violacao(x['Média Anual'],x['Excedências Diárias']), axis = 1)
medicoes_PM10_final

In [ ]:
# Remove valid entries
violacoes_PM10 = medicoes_PM10_final.loc[medicoes_PM10_final['Motivo'] != 'Válido']
violacoes_PM10

In [ ]:
# Save as CSV
violacoes_PM10.to_csv(dir_destination + 'violacoes_pm25.csv', index=False)